In [179]:
import pandas as pd

In [180]:
df = pd.read_csv('binance_api_mediumtune_15min_sRSI-BB.csv')

In [181]:
df

,timestamp,open,high,low,close,Bear,Volatile,Bull,rsi,stochrsi_k,stochrsi_d,bb_upper,bb_middle,bb_lower
0,2023-03-01 13:00:00,23722.60,23735.17,23661.37,23685.87,1,0,0,46.720630,0.078369,0.129473,23842.480815,23758.3325,23674.184185
1,2023-03-01 13:15:00,23685.87,23709.67,23636.50,23646.54,1,0,0,42.247494,0.078369,0.124888,23850.827189,23753.9030,23656.978811
2,2023-03-01 13:30:00,23647.11,23710.43,23616.45,23668.02,1,0,0,45.326263,0.089758,0.122604,23853.257598,23749.4055,23645.553402
3,2023-03-01 13:45:00,23668.02,23703.48,23653.16,23689.45,1,0,0,48.288154,0.112104,0.122151,23832.051384,23740.4890,23648.926616
4,2023-03-01 14:00:00,23689.97,23711.00,23651.09,23688.07,1,0,0,48.107418,0.122106,0.121579,23810.147526,23732.1890,23654.230474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17570,2023-08-31 14:45:00,27120.90,27192.78,27111.17,27175.66,1,0,0,46.794458,0.191883,0.310925,27370.553238,27213.1560,27055.758762
17571,2023-08-31 15:00:00,27175.67,27175.67,27150.00,27174.97,1,0,0,46.719586,0.160950,0.302662,27369.986246,27211.7555,27053.524754
17572,2023-08-31 15:15:00,27174.97,27179.74,27125.75,27144.66,1,0,0,43.432136,0.169270,0.293065,27369.582526,27208.6680,27047.753474
17573,2023-08-31 15:30:00,27144.66,27149.14,26803.73,26915.71,1,0,0,27.621558,0.149383,0.280628,27399.169085,27193.8110,26988.452915


In [182]:
df['change'] = None

for i in range(1, len(df)):
    close = df.iloc[i, 4]
    previous_close = df.iloc[i-1, 4]

    change = close - previous_close
    change_percentage = change * 100 / close
    df.loc[i,'change'] = change_percentage


In [183]:
df = df.iloc[1:,:]
df.reset_index(drop=True, inplace=True)

In [184]:
df

,timestamp,open,high,low,close,Bear,Volatile,Bull,rsi,stochrsi_k,stochrsi_d,bb_upper,bb_middle,bb_lower,change
0,2023-03-01 13:15:00,23685.87,23709.67,23636.50,23646.54,1,0,0,42.247494,0.078369,0.124888,23850.827189,23753.9030,23656.978811,-0.166325
1,2023-03-01 13:30:00,23647.11,23710.43,23616.45,23668.02,1,0,0,45.326263,0.089758,0.122604,23853.257598,23749.4055,23645.553402,0.090755
2,2023-03-01 13:45:00,23668.02,23703.48,23653.16,23689.45,1,0,0,48.288154,0.112104,0.122151,23832.051384,23740.4890,23648.926616,0.090462
3,2023-03-01 14:00:00,23689.97,23711.00,23651.09,23688.07,1,0,0,48.107418,0.122106,0.121579,23810.147526,23732.1890,23654.230474,-0.005826
4,2023-03-01 14:15:00,23688.07,23703.14,23663.00,23685.29,1,0,0,47.719934,0.123680,0.119785,23803.109309,23727.0465,23650.983691,-0.011737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17569,2023-08-31 14:45:00,27120.90,27192.78,27111.17,27175.66,1,0,0,46.794458,0.191883,0.310925,27370.553238,27213.1560,27055.758762,0.201541
17570,2023-08-31 15:00:00,27175.67,27175.67,27150.00,27174.97,1,0,0,46.719586,0.160950,0.302662,27369.986246,27211.7555,27053.524754,-0.002539
17571,2023-08-31 15:15:00,27174.97,27179.74,27125.75,27144.66,1,0,0,43.432136,0.169270,0.293065,27369.582526,27208.6680,27047.753474,-0.111661
17572,2023-08-31 15:30:00,27144.66,27149.14,26803.73,26915.71,1,0,0,27.621558,0.149383,0.280628,27399.169085,27193.8110,26988.452915,-0.850618


In [185]:
for i in range(len(df)):
    if df.loc[i, 'change'] * 100 == 0: print(f"There is a time period (index {i}) with zero change!")

There is a time period (index 2201) with zero change!
There is a time period (index 2202) with zero change!
There is a time period (index 2203) with zero change!
There is a time period (index 2204) with zero change!
There is a time period (index 2205) with zero change!
There is a time period (index 4339) with zero change!
There is a time period (index 12530) with zero change!
There is a time period (index 13063) with zero change!
There is a time period (index 13564) with zero change!
There is a time period (index 13748) with zero change!
There is a time period (index 13906) with zero change!
There is a time period (index 14342) with zero change!
There is a time period (index 15783) with zero change!
There is a time period (index 15791) with zero change!
There is a time period (index 15885) with zero change!
There is a time period (index 16186) with zero change!


In [186]:
df = df[~(df['change'] == 0)].reset_index(drop=True)

In [187]:
# eğer bull bir önceki değerde 1 değilse ve şimdi 1 ise
# eğer bull bir önceki değerde 1 ise ve şimdi 

# df['investment'] = 100 #$
# for i in range(0, len(df)-1):
#     if df.loc[i, 'bull'] == 1:
#         df.loc[i+1, 'investment'] = (df.loc[i, 'investment'] * df.loc[i+1, 'change'] / 100) + df.loc[i-1, 'investment'] #inside
#     elif df.loc[i, 'volatile'] == 1:
#         if df.loc[i, 'investment'] != df.loc[i-1, 'investment']:
#             df.loc[i+1, 'investment'] = (df.loc[i, 'investment'] * df.loc[i+1, 'change'] / 100) + df.loc[i-1, 'investment'] #inside
#         else: df.loc[i+1, 'investment'] = df.loc[i, 'investment'] #not inside
    
#     elif df.loc[i, 'bear'] == 1: df.loc[i+1, 'investment'] = df.loc[i, 'investment'] #not inside

# df.loc[:,['bear', 'volatile', 'bull', 'change', 'investment']]
# print("The investment strategy profits {:.0f}%".format(df.iloc[-1, -1] - df.iloc[0, -1]))

In [188]:
# df['investment'] = 100 #$
# for i in range(0, len(df)-1):
#     if df.loc[i, 'bull'] == 1:
#         df.loc[i+1, 'investment'] = (df.loc[i, 'investment'] * df.loc[i+1, 'change'] / 100) + df.loc[i-1, 'investment'] #inside
#     elif df.loc[i, 'volatile'] == 1: df.loc[i+1, 'investment'] = df.loc[i, 'investment'] #not insid
  
#     elif df.loc[i, 'bear'] == 1: df.loc[i+1, 'investment'] = df.loc[i, 'investment'] #not inside

# df.loc[:,['bear', 'volatile', 'bull', 'change', 'investment']]
# print("The investment strategy profits {:.0f}%".format(df.iloc[-1, -1] - df.iloc[0, -1]))

In [189]:
df.columns.to_series(index=range(len(df.columns)))

0      timestamp
1           open
2           high
3            low
4          close
5           Bear
6       Volatile
7           Bull
8            rsi
9     stochrsi_k
10    stochrsi_d
11      bb_upper
12     bb_middle
13      bb_lower
14        change
dtype: object

In [191]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaled_array = scaler.fit_transform(df.iloc[:,[4,8,9,10,11,12,13]])

scaled_df_X = pd.DataFrame(data = scaled_array, columns=[
 'close',
 'rsi',
 'stochrsi_k',
 'stochrsi_d',
 'bb_upper',
 'bb_middle',
 'bb_lower'])

change_X = df.iloc[:,-1:]

change_X

,change
0,-0.166325
1,0.090755
2,0.090462
3,-0.005826
4,-0.011737
...,...
17553,0.201541
17554,-0.002539
17555,-0.111661
17556,-0.850618


In [192]:
x = pd.concat([scaled_df_X, change_X], axis=1)

In [193]:
df[['Bear', 'Volatile', 'Bull']].value_counts()

Bear  Volatile  Bull
0     1         0       8682
      0         1       4589
1     0         0       4287
dtype: int64

In [194]:
target = df.iloc[:,5:8]
target.reset_index(drop=True, inplace=True)
target

,Bear,Volatile,Bull
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
17553,1,0,0
17554,1,0,0
17555,1,0,0
17556,1,0,0


**How does labeled data perform?** (for train set)

In [195]:
x.values

array([[0.33397918397117676, 0.4193604630538328, 0.06932414055929835,
        ..., 0.3386168061237065, 0.34894132307314263,
        -0.1663245447325406],
       [0.3357615657742865, 0.4535465000024058, 0.08105627455337867, ...,
        0.33822986956646117, 0.34797721194209075, 0.09075537370679745],
       [0.3375397986439628, 0.4864347332878509, 0.10407515583302071, ...,
        0.3374627500972722, 0.3482618540455287, 0.0904622099711065],
       ...,
       [0.6242485244318243, 0.4325144898357268, 0.16296337966188773, ...,
        0.6358430151713979, 0.6350653150317731, -0.11166100441118551],
       [0.6052505582389935, 0.2569570158761003, 0.1424771819581389, ...,
        0.6345648125185916, 0.6300613515370843, -0.8506184678018924],
       [0.6057733238516372, 0.2655341477429325, 0.1320596989540037, ...,
        0.6333611150483476, 0.6262138666246564, 0.0234009273453181]],
      dtype=object)

In [196]:
from myCombinedLSTMClassifier import My_Combined_LSTM_Classifier

myObject1 = My_Combined_LSTM_Classifier(
    x = x.values,
    y = target.values,
    test_num=2000,
    time_steps=10,
    model_activation = 'tanh',
    epoch = 15,
    fit_verbose = 1
)

Epoch 1/15
486/486 [==============================] - 23s 26ms/step - loss: 1.0548 - accuracy: 0.4731
Epoch 2/15
486/486 [==============================] - 15s 31ms/step - loss: 1.0532 - accuracy: 0.4758
Epoch 3/15
486/486 [==============================] - 15s 31ms/step - loss: 1.0511 - accuracy: 0.4767
Epoch 4/15
486/486 [==============================] - 15s 31ms/step - loss: 1.0490 - accuracy: 0.4776
Epoch 5/15
486/486 [==============================] - 15s 31ms/step - loss: 1.0487 - accuracy: 0.4780
Epoch 6/15
486/486 [==============================] - 15s 31ms/step - loss: 1.0423 - accuracy: 0.4815
Epoch 7/15
486/486 [==============================] - 16s 32ms/step - loss: 1.0339 - accuracy: 0.4832
Epoch 8/15
486/486 [==============================] - 16s 32ms/step - loss: 1.0305 - accuracy: 0.4862
Epoch 9/15
486/486 [==============================] - 16s 33ms/step - loss: 1.0266 - accuracy: 0.4869
Epoch 10/15
486/486 [==============================] - 17s 34ms/step - loss: 1.024

**Results without RSI:**

![Alt text](image-1.png)

In [ ]:
myObject1.model_tune(
    test_num = [3000], 
    epoch = [10,20], 
    time_steps = [7,14], 
    units = [50, 100, 150], 
    dropout = [0.25, 0.5], 
    hidden_layer_num = [3,5], 
    model_activation = ['relu', 'tanh'], 
    batch_size = [32,64]
)

THE MODEL IS TUNING!
SEE U AN ETERNITY LATER *_*

Total Progress is creating by 192 steps.



KeyboardInterrupt: 